In [1]:
import pandas as pd
from surprise import SVD, SVDpp, SlopeOne, NMF, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering
from surprise import Dataset
from surprise import Reader
from surprise.model_selection import cross_validate
from surprise import accuracy
import math
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_pickle('C:\\Users\\ineso\\FEUP-3ano\\gulbenkian-ai\\data\\vegas-restaurants\\vegas_review_reduced.pickle')
df = df.sample(n=20000, random_state=1)

In [3]:
df.head()

,user_id,business_id,stars
1729593,O_Tq3eiDZJTnozuEhkbOqQ,QmVF3KJ07PsdroCFwOC_Ow,4
4990953,ih5FXVPfTSIcPzCfyJLsmg,ARohEoxqg8TzA5gQfdGAZw,1
5790810,t1YrELAeIDjS70Y5L3We-Q,6z3cgIE9dJ01wUm43f0yBQ,2
3683340,me7RdUuFmpOqrTIlFlmXtg,Ch3HkwQYv1YKw_FO06vBWA,3
4238038,4TlDa9pRYVOzH_toFuIj7Q,C5azlhXSTOlqVAM3nl95nw,4


In [5]:
new_df = pd.DataFrame(columns=['user_id', 'business_id', 'stars'])
for index, row in df.iterrows():
    if row['stars'] > 3:
        new_df.loc[index] = [row['user_id'], row['business_id'], 1]
    else:
        new_df.loc[index] = [row['user_id'], row['business_id'], 0]

In [6]:
new_df.head()

,user_id,business_id,stars
1729593,O_Tq3eiDZJTnozuEhkbOqQ,QmVF3KJ07PsdroCFwOC_Ow,1
4990953,ih5FXVPfTSIcPzCfyJLsmg,ARohEoxqg8TzA5gQfdGAZw,0
5790810,t1YrELAeIDjS70Y5L3We-Q,6z3cgIE9dJ01wUm43f0yBQ,0
3683340,me7RdUuFmpOqrTIlFlmXtg,Ch3HkwQYv1YKw_FO06vBWA,0
4238038,4TlDa9pRYVOzH_toFuIj7Q,C5azlhXSTOlqVAM3nl95nw,1


In [7]:
trainset, testset = train_test_split(new_df, test_size=0.25)

In [8]:
reader = Reader(rating_scale=(0, 1))
train_data = Dataset.load_from_df(trainset[['user_id', 'business_id', 'stars']], reader)
train_set = train_data.build_full_trainset()

In [22]:
# [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]
algorithm = CoClustering()
algorithm.fit(train_set)

true_positives = 0
false_positives = 0
false_negatives = 0
for index, row in testset.iterrows():
    prediction = algorithm.predict(row['user_id'], row['business_id'], r_ui=row['stars']).est
    if row['stars'] == 1:
        if prediction > 0.5:
            true_positives += 1
        else:
            false_negatives += 1
    else:
        if prediction > 0.5:
            false_positives += 1

precision = true_positives/(true_positives+false_positives)
recall = true_positives/(true_positives+false_negatives)
f1 = 2 * ((precision*recall)/(precision+recall))

print(f1)

0.797404823111764
